#TP 1 : LE MODELE HMC POUR LA SEGMENTATION DE TEXTE

Nom: DOUCOURE

Prénom: Dioula

##**Question 1**: Démontrez que les fonctions forward et backward se calculent bien 

Notation: Probabilité d'un état: $\pi=P(X_t=i)$ \\
Probabilité de transition $a_{ij}=P(X_t=j|X_{t-1}=i)$ \\
Probabilité d'emission: $b_i(y)=P(Y_t=y|X_t=i)$

**Forward** \\
On a $\alpha_i(t) =P(X_t=i,y_{1:t})$ \\
Pour $t=1$, on peut écrire:

\begin{align} 
\alpha_i(1) &= P(X_1= i,y_1) \\ 
                   &= P(y_1|X_1=i)P(X_1=i) \\ 
                   &= P(X_t=i) P(y_1|X_t=i) \quad \quad (X_t \quad uniforme)\\ 
                   &= \pi_i b_{i}(y_1) \\ 
\end{align}                   




Déterminins $\alpha_i(t+1)$ pour $t\in \{2,\dots, T-2\}$, on a:

\begin{align} 
\alpha_i(t+1) &= P \Big( X_{t+1}=i, y_{1:t+1} \Big)  \\ 
                   &={\sum_{j=1}^N} P\Big(X_{t+1}=i,X_t=j,\quad y_{1:t+1} \Big) \\ 
                   &=  \sum_{j=1}^N  P\Big(y_{t+1} | X_{t+1}=i, y_{1:t},X_t= j\Big) 
	                P\Big(X_{t+1}=i, y_{1:t},X_t=j \Big) \\ 
                   &=  \sum_{j=1}^N  P\Big(y_{t+1} | X_{t+1}= i, y_{1:t}, X_t= j\Big) 
	                P\Big(X_{t+1}=i | y_{1:t}, X_t=j\Big) P\Big(X_t=j, y_{1:t}\Big)\\ 
                   &=  \sum_{j=1}^N  P\Big(y_{t+1} | X_{t+1}= i\Big) P\Big(X_{t+1}=i | X_t=j\Big) p\Big(X_t=j, y_{1:t}\Big)\\ 
                   &= P\Big(y_{t+1} | X_{t+1}=i\Big) \sum_{j=1}^N P\Big(X_{t+1}=i | X_t=j\Big) P\Big(X_t=j, y_{1:t}\Big) \\ 
                   &= b_{i}(y_{t+1})\sum_{j=1}^N a_{ji} \alpha_i(t)
\end{align}


**Backward** \\
Avec les mêmes notations que pour le forward, on a:

\begin{align} 
\beta_i(t) &= P \Big( y_{t+1:T} | X_t = i \Big) \\ 
&= \sum_{j=1}^T P\Big( y_{t+1:T}, X_{t+1}= j | X_t = i  \Big)        \\ 
&= \sum_{j=1}^T P\Big( y_{t+2:T} |  y_{t+1} , X_{t+1} = j , X_t = i  \Big) 
P \Big( y_{t+1} , X_{t+1} = j | X_t = i  \Big) \\ 
&= \sum_{j=1}^T P\Big( y_{t+2:T}  |  y_{t+1} , X_{t+1} = j , X_t = i  \Big) 
		P \Big( y_{t+1}|  X_{t+1} = j , X_t = i  \Big) P \Big( X_{t+1} = j | X_t = i  \Big) \\ 
&= \sum_{j=1}^T  P\Big( y_{t+2:T} |  X_{t+1} = j  \Big) P \Big( y_{t+1} |  X_{t+1} = j \Big)  P \Big( X_{t+1} = j | X_t = i  \Big) \\ 
&= \sum_{j=1}^T \beta_j(t+1) b_{j}(y_{t+1}) a_{ij} \\ 
\end{align}



##**Question 2.1** Description du fonctionnement de la fonction d'estimation des paramètres $\pi$, A et B.

In [ ]:
import numpy as np


def HMC_Parameters_Dict(dataset):
        
    Pi = {}
    A = {}
    B = {}
        
    for Z in dataset:
        
        Z0 = Z[0]
        X0 = Z0[0]
        Y0 = Z0[1]
        
        if X0 not in Pi.keys():
            Pi[X0] = 0
            
        Pi[X0] = Pi[X0] + 1
        
        if X0 not in B.keys():
            B[X0] = {}
            
        if Y0 not in B[X0].keys():
            B[X0][Y0] = 0
        
        B[X0][Y0] = B[X0][Y0] + 1
        
        
        Zi_prev = Z0
        
        for Zi in Z[1:]:
            x = Zi[0]
            y = Zi[1]
            
            if x not in Pi.keys():
                Pi[x] = 0
            
            if Zi_prev[0] not in A.keys():
                A[Zi_prev[0]] = {}
                
            if x not in A[Zi_prev[0]].keys():
                A[Zi_prev[0]][x] = 0
                
            if x not in B.keys():
                B[x] = {}
                
            if y not in B[x].keys():
                B[x][y] = 0
                            
            Pi[x] = Pi[x] + 1
            A[Zi_prev[0]][x] = A[Zi_prev[0]][x] + 1
            B[x][y] = B[x][y] + 1
            
            Zi_prev = Zi
            
    sum_Pi = np.sum(list(Pi.values()))
    for key in Pi.keys():
        Pi[key] = Pi[key]/sum_Pi
        
    for key_A_1 in A.keys():
        sum_A = np.sum(list(A[key_A_1].values()))
        for key_A_2 in A[key_A_1].keys():
            A[key_A_1][key_A_2] = A[key_A_1][key_A_2]/sum_A
            
    for key_B_1 in B.keys():
        sum_B = np.sum(list(B[key_B_1].values()))
        for key_B_2 in B[key_B_1].keys():
            B[key_B_1][key_B_2] = B[key_B_1][key_B_2]/sum_B
    
    
    return Pi, A, B

La fonction ci-dessus prend en paramètre notre dataset qui se présente sous la forme d'une liste python. Afin de simplifier la description, nous allons prenons l'exemple de dataset ci-dessous pour bien expliquer le fonctionnement de notre fonction. $Dataset=[(Noun, Bruce), (Verb, is),(Noun, home)]$.

On commence par initialiser nos paramètres $\pi$, A et B, qui sont des dictionnaires (vides au début).  On parcourt notre dataset avec la boucle for. On définit Z0 comme étant le premier élément de notre dataset ie Z0 est de la forme ($x_1$, $y_1$) où $x_1$ désigne la fonction grammaticale et $y_1$ la variable observée. Avec notre exemple précédent Z0 sera égale à $(Noun, Bruce)$. On affecte ensuite à X0 la fonction grammaticale $x_1$ et à Y0 $y_1$. On vérifie ensuite si la fonction grammaticale X0 ne fait pas partie des clés du dictionnaire $\pi$. Si c'est le cas, on ajoute cette clé à ce dictionnaire avec la valeur 0. ($\{"Noun": 0\}$ avec notre exemple illustratif). 
On incrémente la valeur de clé X0 de $\pi$, pour dire que ce dernier apparaît au moins une fois..

Si la fonction grammitacale X0 n'est pas une clé du dictionnaire B, on l'ajoute avec comme valeur un dictionnaire vide. B est donc un dictionnaire de dictionnaires c'est à dire de la forme $\{"key1": \{"key1bis": value\}, \dots\}$. Value désigne le nombre d'occurences de la clé key1bis. Si le mot observé Y0 n'est pas une clé du dictionnaire correspondant à la valeur de la clé X0 ie une clé du dictionnaire suivant: $\{"key1bis": value\}$, alors on l'ajoute avec une entrée qui vaut 0. Dans notre exemple $value=0$ et B est de la forme : $\{"X0": \{"Y0": 0\}, \dots\}=\{"Noun": \{"Bruce": 0\}, \dots\}$. On incrémente ensuite pour dire que Y0 apparaît au moins une fois.

Le dictionnaire A est aussi un dictionnaire de dictionnaire qui contient des fonctions grammaticales et dans chaque dictionnaire de fonction grammaticale on a une fonction grammaticale et son nombre d'apparition. On affecte à Z_prev Z0 et parcourt à nouveau notre dataset comme précédemment. On définit $x$ et $y$ comme étant respectivement la fonction grammaticale et le mot observé. Si x n'existe pas dans $\pi$, on le crée avec un nombre d'occurences qui vaut zéro. Dans A, on crée un dictionnaire de la foncton grammaticale précédente et on vérifie si x apparait dans dictionnaire correspondant à la valeur de cette entrée. Si ce n'est pas le cas, on le crée comme dans ça été fait un peu plus haut. 

A la fin, on normalise afin que les valeurs soient comprises entre 0 et 1.

##**Question 2.2** 

On préfère utiliser les dictionnaires à la place des matrices car la recherche y est beaucoup plus rapide. De plus, les matrices prennent beaucoup de mémoire. Par conséquent pour éviter gaspiller de la mémoire, on utilise les dictionnaires.

##**Question 3** Complétion des codes du fichier HMC_Forword_Backword_Restator.py 

In [ ]:
import numpy as np

class HMC_Forward_Backward_Restorator:
    
    def __init__(self, hmc = False):
        self.hmc = hmc
        
    def forward_prime(self, Omega_X, Y, parameter_set, default = 0):
        T = len(Y)
        alpha = np.zeros((T, len(Omega_X)))
        
        Pi = parameter_set["Pi"]
        A = parameter_set["A"]
        B = parameter_set["B"]

        alpha[0] = self.compute_alpha_1(Omega_X, Y, Pi, B, default)
        
        for t in range(0, T - 1):
            alpha[t + 1] = self.compute_alpha_t_plus_1(Omega_X, Y, t + 1, A, B, alpha[t], default)
     
        return alpha
    
    
    def backward_prime(self, Omega_X, Y, parameter_set, default = 0):
        T = len(Y)
        beta = np.zeros((T, len(Omega_X)))
        
        A = parameter_set["A"]
        B = parameter_set["B"]
        
        beta[T - 1] = self.compute_beta_T(Omega_X)
        
        for t in range(T - 2, -1, -1):
            beta[t] = self.compute_beta_t(Omega_X, Y, t + 1, A, B, beta[t + 1], default)
        
        return beta
    
    
    def compute_gamma(self, alpha, beta):
        gamma = alpha * beta
    
        gamma = np.transpose(gamma)/np.sum(gamma, axis = 1)
        gamma = np.transpose(gamma)
    
        return gamma
    
    def return_gamma(self, Omega_X, Y, parameter_set, epsilon_laplace = 0):
        alpha = self.forward_prime(Omega_X, Y, parameter_set, epsilon_laplace)
        beta = self.backward_prime(Omega_X, Y, parameter_set, epsilon_laplace)  
        
        gamma = self.compute_gamma(alpha, beta)
        
        return gamma
    
    def restore_X(self, Omega_X, Y, parameter_set, epsilon_laplace = 0):
        alpha = self.forward_prime(Omega_X, Y, parameter_set, epsilon_laplace)
        beta = self.backward_prime(Omega_X, Y, parameter_set, epsilon_laplace)  
        
        gamma = self.compute_gamma(alpha, beta)
        
        X = []
        indexes = np.argmax(gamma, axis = 1)
        
        for index in indexes:
            X.append(Omega_X[index])
        
        return X
    
    
    #######################
    ### COMPUTING ALPHA ###
    #######################
    
    def compute_alpha_1(self, Omega_X, Y, Pi, B, default = 0):
      
        alpha_1 = np.zeros(len(Omega_X))
                
        for idi, i in enumerate(Omega_X):
            if Y[0] in B[i]:
                alpha_1[idi]= B[i][Y[0]]*Pi[i]
                    
        if np.sum(alpha_1) == 0:
            for idi, i in enumerate(Omega_X):
                alpha_1[idi]= Pi[i]
            
        alpha_1 = (alpha_1 + default)/np.sum(alpha_1 + default)
                    
        return alpha_1
    
    def compute_alpha_t_plus_1(self, Omega_X, Y, t_plus_1, A, B, alpha_t, default = 0):
        alpha_t_plus_1 = np.zeros(len(Omega_X))
        
        yt_plus_1 = Y[t_plus_1]
                
        for idi, i in enumerate(Omega_X):
            for idj, j in enumerate(Omega_X):
            
                if j in A and i in B:
                    if i in A[j] and yt_plus_1 in B[i]:
                        alpha_t_plus_1[idi] += B[i][yt_plus_1]*alpha_t[idj]*A[j][i]
        
        
        if np.sum(alpha_t_plus_1) == 0:
            for idi, i in enumerate(Omega_X):
                for idj, j in enumerate(Omega_X):
                    
                    if j in A:
                        if i in A[j]:
                            alpha_t_plus_1[idi] += alpha_t[idj]*A[j][i]
        

        alpha_t_plus_1 = (alpha_t_plus_1 + default)/np.sum(alpha_t_plus_1 + default)
        return alpha_t_plus_1
    
    
    ######################
    ### COMPUTING BETA ###
    ######################
    
    def compute_beta_T(self, Omega_X):
        beta_T = np.ones(len(Omega_X))
        beta_T = beta_T/np.sum(beta_T)
        
        return beta_T
    
    
    def compute_beta_t(self, Omega_X, Y, t_plus_1, A, B, beta_t_plus_1, default = 0):
        beta_t = np.zeros(len(Omega_X))
        
        yt_plus_1 = Y[t_plus_1]
        
        for idi, i in enumerate(Omega_X):
            for idj, j in enumerate(Omega_X):
                if i in A and j in B:
                    if j in A[i] and yt_plus_1 in B[j]:
                        beta_t[idi] +=  beta_t_plus_1[idj]*A[i][j]*B[j][yt_plus_1]   
                            
        if np.sum(beta_t) == 0:
            for idi, i in enumerate(Omega_X):
                for idj, j in enumerate(Omega_X):
                    if i in A:
                        if j in A[i]:
                            beta_t[idi] += beta_t_plus_1[idj]*A[i][j]
        
        
        beta_t = (beta_t + default)/np.sum(beta_t + default)
        return beta_t
        


##**Question 4** Résultats et Commentaires

Nous obtenons les résultats ci-dessous. \\

\begin{array}{|l|l|l|l|l|} 
\hline
                       & ~ ~ ~ ~ \textit{Accuracy} & ~ ~ \textit{Known Words} & ~~ \textit{Unknown Words} & ~ \textit{Training time}  \\ 
\hline
CoNLL2000 POS Tagging  & 5.359140511218513         & 1.9444129325014217       & 50.9388249545572986       & 1.0881872177124023        \\ 
\hline
CoNLL 2003 POS Tagging & 9.912919774108715         & 3.979520031059664        & 57.09048811428516         &                           \\ 
\hline
UD English POS Tagging & 11.161141217723937        & 6.222559767491413        & 65.26917579799905         &                           \\ 
\hline
CoNLL 2000 Chunking    & 7.191253139709147         & 6.677254679523543        & 14.052789642640823        & 1.0331740379333496        \\ 
\hline
CoNLL 2003 Chunking    & 5.62357201362245          & 5.3073475528837831       & 8.9330503356939135        & 1.3438026905059814        \\ 
\hline
HMC F1 CoNLL 2003 Ner  & 0.7249928346231013        & 0.8692506459948321       & 0.011082693947144074      &                           \\
\hline
\end{array}


La première observation qu'on peut faire en voyant ce tableau est la différence significative des résultats obtenus pour les mots connus et ceux qui sont inconnus. En effet, on remarque que pour les mots qui ne sont pas présents dans notre dataset d'entrainement (ie inconnus), le taux d'erreur est largement plus grand que celui pour les mots connus. Une explication assez logique à cette différence est que la connaissance d'un mot c'est-à-dire sa présence dans le dataset nous fournit plus d'informations sur les fonctions grammaticales donc sur l'état caché, ce qui n'est pas le cas pour un mot inconnu.

Une comparaison entre le Chunking et Pos Tagging montre que le premier présente des résultats plus satisfaisants. Cette performance pourrait s'expliquer par le fait que le chunking comme on l'a vu dans la partie théorique de ce TP est à un cran au dessus du Pos Tagging. En effet, dans le POS Tagging, on labélise chaque mot d'une phrase alors que pour le chunking on s'intéresse au groupe de mots. Pour des mots composés comme "South Africa" par exemple, le Pos Tagging peut donner lieu à des erreurs. 

La performance du chunking est plus intéressante pour les mots inconnus. A tire d'exemple, on a une erreur d'environ 51% pour les mots inconnus dans le cas du Pos Tagging (CoNLL 2000) contre un taux d'erreur de de 14%.

##**Question 5** Normalisation

On normalise pour éviter certaines erreurs qui peuvent être dues à des faibles valeurs des probas forward et backward. La normalisation permet donc de se prémunir de ces erreurs. Elle n'affecte pas les résultats de notre algorithme. Considérons la formule ci-dessous:

\begin{align}
P(X_t=i|y_{1:T})=\frac{\alpha_i(t)\beta_i(t)}{\sum_{j \in \Omega_X}\alpha_j(t)\beta_j(t)}
\end{align}
Normaliser les probas forward et backward revient à diviser le numérateur et le dénominateur des probabilités à posteriori par la même valeur, ce qui ne change rien à cette quantité.

##**Question 6** Utilité de epsilon_laplace

Ce paramètre sert à mesurer la confiance qu'on a des résultats que nous avons trouvés pour les calculs de nos probabilités forward et backward. Dans notre code, il est égale à 0, ce qui se traduit par une confiance $\textit{"absolue"}$ en ceux-ci.  

##**Question 7** Amélioration des performances de notre algorithme.

In [ ]:
import numpy as np

class HMC_Down_Forward_Backward_Restorator:
    
    def __init__(self, hmc = False):
        self.hmc = hmc
        
    def forward_prime(self, Omega_X, list_Y, parameter_set, default = 0):
        T = len(list_Y[0])
        alpha = np.zeros((T, len(Omega_X)))
        
        Pi_hmc = parameter_set["Pi_HMC"]
        A_hmc = parameter_set["A_HMC"]
        list_B_hmc = parameter_set["list_B_HMC"]

        alpha[0] = self.compute_alpha_1(Omega_X, list_Y, Pi_hmc, list_B_hmc, default)
        
        for t in range(0, T - 1):
            alpha[t + 1] = self.compute_alpha_t_plus_1(Omega_X, list_Y, t + 1, A_hmc, list_B_hmc, alpha[t], default)
     
        return alpha
    
    
    def backward_prime(self, Omega_X, list_Y, parameter_set, default = 0):
        T = len(list_Y[0])
        beta = np.zeros((T, len(Omega_X)))
        
        A_hmc = parameter_set["A_HMC"]
        list_B_hmc = parameter_set["list_B_HMC"]
        
        beta[T - 1] = self.compute_beta_T(Omega_X)
        
        for t in range(T - 2, -1, -1):
            beta[t] = self.compute_beta_t(Omega_X, list_Y, t + 1, A_hmc, list_B_hmc, beta[t + 1], default)
        
        return beta
    
    
    def compute_gamma(self, alpha, beta):
        gamma = alpha * beta
    
        gamma = np.transpose(gamma)/np.sum(gamma, axis = 1)
        gamma = np.transpose(gamma)
    
        return gamma
    
    def return_gamma(self, Omega_X, Y, parameter_set, epsilon_laplace = 0):
        alpha = self.forward_prime(Omega_X, Y, parameter_set, epsilon_laplace)
        beta = self.backward_prime(Omega_X, Y, parameter_set, epsilon_laplace)  
        
        gamma = self.compute_gamma(alpha, beta)
        
        return gamma
    
    def restore_X(self, Omega_X, Y, parameter_set, epsilon_laplace = 0):
        alpha = self.forward_prime(Omega_X, Y, parameter_set, epsilon_laplace)
        beta = self.backward_prime(Omega_X, Y, parameter_set, epsilon_laplace)  
        
        gamma = self.compute_gamma(alpha, beta)
        
        X = []
        indexes = np.argmax(gamma, axis = 1)
        
        for index in indexes:
            X.append(Omega_X[index])
        
        return X
    
    
    #######################
    ### COMPUTING ALPHA ###
    #######################
    
    def compute_alpha_1(self, Omega_X, list_Y, Pi_hmc, list_B_hmc, default = 0):
        alpha_1 = np.zeros(len(Omega_X))
        for idi, i in enumerate(Omega_X):
            if list_Y[0][0] in list_B_hmc[0][i]:
                alpha_1[idi]  = Pi_hmc[i]*list_B_hmc[0][i][list_Y[0][0]]
                
        
        if np.sum(alpha_1) == 0:
             for idi, i in enumerate(Omega_X):
                
                 if list_Y[1][0] in list_B_hmc[1][i]:
                     alpha_1[idi] = Pi_hmc[i]*list_B_hmc[1][i][list_Y[1][0]]
                     
        if np.sum(alpha_1) == 0:
            for idi, i in enumerate(Omega_X):
               alpha_1[idi]=Pi_hmc[i]
            
        # to complete
            
        alpha_1 = (alpha_1 + default)/np.sum(alpha_1 + default)
                    
        return alpha_1
    
    def compute_alpha_t_plus_1(self, Omega_X, list_Y, t_plus_1, A_hmc, list_B_hmc, alpha_t, default = 0):
        alpha_t_plus_1 = np.zeros(len(Omega_X))
        
        
        yt_plus_1 = list_Y[0][t_plus_1]
        yft_plus_1 = list_Y[1][t_plus_1]
            
        for idi, i in enumerate(Omega_X):
            for idj, j in enumerate(Omega_X):
            
                if j in A_hmc and i in list_B_hmc[0]:
                    if i in A_hmc[j] and yt_plus_1 in list_B_hmc[0][i]:
                        alpha_t_plus_1[idi] += list_B_hmc[0][i][yt_plus_1]*A_hmc[j][i]*alpha_t[idj]
                        
        if np.sum(alpha_t_plus_1) == 0:
             
             for idi, i in enumerate(Omega_X):
                 for idj, j in enumerate(Omega_X):
                     if j in A_hmc and i in list_B_hmc[1]:
                            if i in A_hmc[j] and yft_plus_1 in list_B_hmc[1][i]:
                                alpha_t_plus_1[idi] += list_B_hmc[1][i][yft_plus_1]*A_hmc[j][i]*alpha_t[idj]
    
        if np.sum(alpha_t_plus_1) == 0:
            for idi, i in enumerate(Omega_X):
                for idj, j in enumerate(Omega_X):
                    
                    if j in A_hmc:
                        if i in A_hmc[j]:
                            alpha_t_plus_1[idi] += A_hmc[j][i]*alpha_t[idj]
        
        
        # to complete

        alpha_t_plus_1 = (alpha_t_plus_1 + default)/np.sum(alpha_t_plus_1 + default)
        return alpha_t_plus_1
    
    
    ######################
    ### COMPUTING BETA ###
    ######################
    
    def compute_beta_T(self, Omega_X):
        # to complete
        beta_T = np.ones(len(Omega_X)) 
        beta_T = beta_T/np.sum(beta_T)
        return beta_T
    
    
    def compute_beta_t(self, Omega_X, list_Y, t_plus_1, A_hmc, list_B_hmc, beta_t_plus_1, default = 0):
        beta_t = np.zeros(len(Omega_X))
        
        yt_plus_1 = list_Y[0][t_plus_1]
        yft_plus_1 = list_Y[1][t_plus_1]
        
        for idi, i in enumerate(Omega_X):
            for idj, j in enumerate(Omega_X):
                # to complete
                if i in A_hmc and j in list_B_hmc[0]:
                    if j in A_hmc[i] and yt_plus_1 in list_B_hmc[0][j]:
                         beta_t[idi] += beta_t_plus_1[idj]*A_hmc[i][j]*list_B_hmc[0][j][yt_plus_1]
                         
        if np.sum(beta_t) == 0:
             for idi, i in enumerate(Omega_X):
                 for idj, j in enumerate(Omega_X):
                     if i in A_hmc and j in list_B_hmc[1]:
                         if j in A_hmc[i] and yft_plus_1 in list_B_hmc[1][j]:
                             beta_t[idi] += beta_t_plus_1[idj]*A_hmc[i][j]*list_B_hmc[1][j][yft_plus_1]
                                  
        if np.sum(beta_t) == 0:
            for idi, i in enumerate(Omega_X):
                for idj, j in enumerate(Omega_X):
                    # to complete
                    if i in A_hmc :
                        if j in A_hmc[i]:
                            beta_t[idi] += beta_t_plus_1[idj]*A_hmc[i][j]   
        
        # to complete
        beta_t = (beta_t + default)/np.sum(beta_t + default)
        return beta_t
        


###Résultas

\begin{array}{|l|l|l|l|l|} 
\hline
                       & ~ ~ ~ ~ \textit{Accuracy} & ~ ~ \textit{Known Words} & ~~ \textit{Unknown Words} & ~ \textit{Training time}  \\ 
\hline
CoNLL2000 POS Tagging  & 3.4278236274985687         & 1.9353374929098095       & 23.34948516050879       & 2.2617568969726562        \\ 
\hline
CoNLL 2003 POS Tagging &  6.2464973919041284         & 4.035330372958683        & 23.827898900250815         &  2.8601086139678955                       \\ 
\hline
UD English POS Tagging & 8.997449792795663       & 6.139931295386361        & 40.30490709861839         &  2.6570358276367188                         \\ 
\hline
CoNLL 2000 Chunking    & 7.0878274268104775         & 6.679523539421439        & 12.537855844942456       &         \\ 
\hline
CoNLL 2003 Chunking    & 5.5028667500107815          & 5.209774089442149       &  7.83330117692455        & 2.5722577571868896       \\ 
\hline
HMC F1 CoNLL 2003 Ner  & 0.7773382671940794        & 0.8708512467755803        & 0.5084075173095944     &  2.5655274391174316                      \\
\hline
\end{array}

Nous obtenons de meilleurs à résultats à cette question et ce pour tous les tests. On constate ici que le taux d'erreur pour le mots inconnus a considérablement diminué. En effet, nous sommes passés respectivement de 50.93%, 57.09% et 65.26% à 23.34%, 23.82% et 40% pour CoNLL2000PosTagging, CoNLL2003PosTagging et UD_English_PosTagging. La prise en compte des $\textit{features}$ a ainsi permis d'améliorer de manière très significative les performances de notre modèle. En effet, nous avons pu obtenir des résultats satisfaisants pour des mots ne sont trouvant pas dans notre dataset d'entrainement.